In [1]:
%load_ext autoreload
%autoreload 2

import ase.io
import numpy as np

import rascaline
import equistore
from equistore import Labels, TensorBlock, TensorMap
import chemiscope

# from rascaline.utils import clebsch_gordan
import clebsch_gordan
import old_clebsch_gordan
import rotations

In [2]:
frames = ase.io.read("combined_magres_spherical.xyz", ":")

# Define hyperparameters for generating the rascaline SphericalExpansion
rascal_hypers = {
    "cutoff": 3.0,  # Angstrom
    "max_radial": 6,  # Exclusive
    "max_angular": 5,  # Inclusive
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}

# chemiscope.show(frames, mode="structure")

# Equivariance Test - SO(3) for $\nu=3$

In [3]:
# Define target lambda channels
lambdas = np.array([0, 1, 2])

# Pick a test frame
frame = frames[0]

# Generate Wigner-D matrices, initialized with random angles
wig = rotations.WignerDReal(lmax=rascal_hypers["max_angular"])
print("Random rotation angles (rad):", wig.angles)

# Randomly rigidly rotate the frame
frame_so3 = rotations.transform_frame_so3(frame, wig.angles)
assert not np.allclose(frame.positions, frame_so3.positions)

# Generate nu=3 descriptor for both frames
nu3 = clebsch_gordan.n_body_iteration_single_center(
    [frame],
    rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    only_keep_parity=+1,
)

nu3_rot = clebsch_gordan.n_body_iteration_single_center(
    [frame_so3],
    rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    only_keep_parity=+1,
)

# Rotate the lambda-SOAP descriptor of the unrotated frame
nu3_unrot_rot = wig.transform_tensormap_so3(nu3)

# Check for equivariance!
assert equistore.equal_metadata(nu3_unrot_rot, nu3_rot)
assert equistore.allclose(nu3_unrot_rot, nu3_rot)
print("SO(3) EQUIVARIANT!")

# chemiscope.show([frame, frame_so3], mode="structure")

Random rotation angles (rad): [2.71801454 5.20220183 3.836667  ]
SO(3) EQUIVARIANT!


# Equivariance Test - O(3) on $\nu=2$ (i.e. $\lambda$-SOAP)

In [4]:
# Define target lambda channels
lambdas = np.array([0, 1, 2, 3, 4, 5])

# Pick a test frame
frame = frames[0]

# Generate Wigner-D matrices, initialized with random angles
wig = rotations.WignerDReal(lmax=rascal_hypers["max_angular"])
print("Random rotation angles (rad):", wig.angles)

# Apply an O(3) transformation to the frame
frame_o3 = rotations.transform_frame_o3(frame, wig.angles)
assert not np.allclose(frame.positions, frame_o3.positions)

# Generate lambda-SOAP for both frames
lsoap = clebsch_gordan.lambda_soap_vector(
    [frame],
    rascal_hypers,
    lambdas=lambdas,
    only_keep_parity=+1,
)

lsoap_o3 = clebsch_gordan.lambda_soap_vector(
    [frame_o3],
    rascal_hypers,
    lambdas=lambdas,
    only_keep_parity=+1,
)

# Apply the O(3) transformation to the TensorMap
lsoap_transformed = wig.transform_tensormap_o3(lsoap)

# Check for equivariance!
assert equistore.equal_metadata(lsoap_transformed, lsoap_o3)
assert equistore.allclose(lsoap_transformed, lsoap_o3)
print("O(3) EQUIVARIANT!")

# chemiscope.show([frame, frame_o3], mode="structure")

Random rotation angles (rad): [3.6769499  0.25087312 3.97576283]
O(3) EQUIVARIANT!


# Old v new lambda-SOAP

In [5]:
# Define target lambda channels
lambdas = np.arange(6)
lambdas

array([0, 1, 2, 3, 4, 5])

In [6]:
lsoap_old = old_clebsch_gordan.lambda_soap_vector(
    frames,
    rascal_hypers,
    lambdas=lambdas,
    lambda_cut=8,
)
lsoap_old

TensorMap with 33 blocks
keys: inversion_sigma  spherical_harmonics_l  species_center
             1                   0                  3
             1                   1                  3
                               ...
            -1                   4                  22
            -1                   5                  22

In [10]:
lsoap_new = clebsch_gordan.lambda_soap_vector(
    frames,
    rascal_hypers,
    lambdas=lambdas,
    lambda_cut=8,
    only_keep_parity=+1,
)
lsoap_new

TensorMap with 18 blocks
keys: spherical_harmonics_l  species_center
                0                  3
                1                  3
                      ...
                4                  22
                5                  22

# Dense v Sparse

In [ ]:
# Dense
lsoap_new0 = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
    lambda_cut=5,
    use_sparse=False,
)
lsoap_new0

In [ ]:
# Sparse
lsoap_new1 = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
    lambda_cut=5,
    use_sparse=True,
    only_keep_parity=+1,
)
lsoap_new1

In [ ]:
# Check sparse == dense
equistore.allclose(lsoap_new0, lsoap_new1)

# Test system 1

In [ ]:
%%timeit

use_sparse = False

tensor_dense = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=use_sparse,
)
tensor_dense

# timeit comes out at about 22 seconds for nu_target = 3

In [ ]:
%%timeit

use_sparse = True

tensor_sparse = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=use_sparse,
)
tensor_sparse

# timeit comes out at about 13 seconds for nu_target = 3

In [ ]:
tensor_sparse = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=True,
)

tensor_dense = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=False,
)

assert equistore.allclose(tensor_dense, tensor_sparse)

# Test system 2

In [ ]:
frames = [ase.io.read("frame.xyz")]

lambdas = np.array([0, 2])

rascal_hypers = {
    "cutoff": 3.0,  # Angstrom
    "max_radial": 6,  # Exclusive
    "max_angular": 5,  # Inclusive
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}

n_body = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
)
n_body